In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

In [ ]:
def load_pce_data():
    """
    Load PyApprox PCE Sobol indices from NPZ files.
    Returns:
    --------
    dict with keys for each PCE max order (4, 6, 8):
        Each contains:
            'E_main': array of main indices for E
            'E_total': array of total indices for E
            'nu_main': array of main indices for nu
            'nu_total': array of total indices for nu
            'samples': array of sample sizes
    """
    # PCE max orders and sample sizes
    pce_orders = [4, 6, 8]
    sample_sizes = [20, 50, 100, 500, 1000, 5000, 10000]

    data = {}

    for order in pce_orders:
        order_data = {
            'samples': [],
            'E_main': [],
            'E_total': [],
            'nu_main': [],
            'nu_total': []
        }

        for n in sample_sizes:
            filepath = f'./data_files/pyapprox_sobol_pce_output/pyapprox_pce_{order}_samples_{n}.npz'

            try:
                npz_data = np.load(filepath)

                # main_effects shape: (3, 3), row 0 = rho, row 1 = E, row 2 = nu
                # columns: col 0 = E_effective, col 1 = stress, col 2 = max_von_Mises_stress
                # total_effects shape: (3, 3), same layout

                # Extract for max_von_Mises_stress (column index 2)
                order_data['samples'].append(n)
                order_data['E_main'].append(npz_data['main_effects'][1, 2])
                order_data['E_total'].append(npz_data['total_effects'][1, 2])
                order_data['nu_main'].append(npz_data['main_effects'][2, 2])
                order_data['nu_total'].append(npz_data['total_effects'][2, 2])

            except FileNotFoundError:
                print(f"Warning: File not found: {filepath}")
                continue

        # Convert lists to numpy arrays
        for key in ['E_main', 'E_total', 'nu_main', 'nu_total']:
            order_data[key] = np.array(order_data[key])
        order_data['samples'] = np.array(order_data['samples'])

        data[order] = order_data

    return data

In [ ]:
pce_data = load_pce_data()

In [ ]:
def create_plot1_E_indices(pce_data):
    """
    Plot E parameter Sobol indices (main and total) vs number of training samples.
    Different PCE max orders shown with different viridis colors.

    Lines (6 total):
    1. E main - PCE order 4 (viridis color 1, solid)
    2. E total - PCE order 4 (viridis color 1, dashed)
    3. E main - PCE order 6 (viridis color 2, solid)
    4. E total - PCE order 6 (viridis color 2, dashed)
    5. E main - PCE order 8 (viridis color 3, solid)
    6. E total - PCE order 8 (viridis color 3, dashed)
    """
    fig, ax = plt.subplots(figsize=(10, 6))

    # Use viridis colormap for different PCE orders
    pce_orders = [4, 6, 8]
    colors_E = cm.viridis(np.linspace(0.05, 0.5, len(pce_orders)))
    colors_nu = cm.viridis(np.linspace(0.55, 1.0, len(pce_orders)))

    for idx, order in enumerate(pce_orders):
        data = pce_data[order]
        color_E = colors_E[idx]

        # Main indices - solid line
        ax.plot(data['samples'], data['E_main'],
                color=color_E, linestyle='-', marker='o', markersize=6,
                label=f'E main (PCE order {order})', linewidth=2)

        # Total indices - dashed line
        ax.plot(data['samples'], data['E_total'],
                color=color_E, linestyle='--', marker='s', markersize=6,
                label=f'E total (PCE order {order})', linewidth=2)

        color_nu = colors_nu[idx]

        # Main indices - solid line
        ax.plot(data['samples'], data['nu_main'],
                color=color_nu, linestyle='-', marker='o', markersize=6,
                label=f'nu main (PCE order {order})', linewidth=2)

        # Total indices - dashed line
        ax.plot(data['samples'], data['nu_total'],
                color=color_nu, linestyle='--', marker='s', markersize=6,
                label=f'nu total (PCE order {order})', linewidth=2)



    ax.set_xscale('log')
    ax.set_xlabel('Number of Training Samples', fontsize=18)
    ax.set_ylabel('Sobol Index Value', fontsize=18)
    ax.set_title('Sobol Indices for max_von_Mises_stress: E Parameter (PCE)', fontsize=18)
    ax.legend(loc='center right', fontsize=12, ncol=2)
    ax.tick_params(axis='both', labelsize=14)
    ax.grid(True, alpha=0.3)
    ax.set_ylim(-0.1, 1.1)
    #ax.ticklabel_format(useOffset=False, style='plain')
    #ax.hlines(1.0, 10, 20000, colors="black", linewidth=1, alpha=0.5)

    plt.tight_layout()
    #plt.savefig('sobol_pce_E_indices.png', dpi=300, bbox_inches='tight')
    #plt.savefig('sobol_pce_E_indices.pdf', bbox_inches='tight')
    #plt.close()

In [ ]:
create_plot1_E_indices(pce_data)

In [ ]:
pce_data